## Semi-mechanistic modelling exploration
Thinking about this equation in Faria, et al:
$\\i_{s,t} = (1-\frac{n_{s,t}}{N})R_{s,t}\sum_{\tau<t} i_{s,\tau}g_{t-\tau}$

For me, this is a more standard "semi-mechanistic" modelling approach,
in that the population is not explicitly partitioned into categories or compartments.
It is partitioned in this way for our standard compartmental models,
including both standard SEIR `summer` models, 
as well as Romain's semi-mechanistic models,
which are compartmental with an additional non-mechanistic random walk 
flow adjustment.

First, ignoring strains, I'll consider:
$\\i_t = (1-\frac{n_t}{N})R_t\sum_{\tau<t} i_{\tau}g_{t-\tau}$

For now, I'll also ignore susceptible depletion and a varying reproduction number, and so consider:
$\\i_t = R_0\sum_{\tau<t} i_\tau g_{t-\tau}$

This notebook builds up the basic code from the first principles,
checking with each extension that the results we are getting back are the same
as in the previous, more explicit version.

In [ ]:
from typing import Dict
from scipy.stats import gamma
import numpy as np
import pandas as pd
pd.options.plotting.backend = 'plotly'

Get some sort of distribution we can sensibly use for the generation time

In [ ]:
def get_gamma_params_from_mean_sd(req_mean: float, req_sd: float) -> Dict[str, float]:
    var = req_sd ** 2.0
    scale = var / req_mean
    a = req_mean / scale
    return {'a': a, 'scale': scale}

In [ ]:
# Model parameters
times = 20
incidence = np.zeros(times)
seed = 1.0
incidence[0] = seed
r0 = 2.0

In [ ]:
# Generation time parameters
req_sd = 1.5
req_mean = 5.0
gamma_params = get_gamma_params_from_mean_sd(req_mean, req_sd)
gen_time_densities = np.diff(gamma.cdf(range(times + 1), **gamma_params))

In [ ]:
pd.Series(gen_time_densities, index=range(times)).plot()

Looping in Python, with pre-calculated generation times

In [ ]:
for t in range(1, times):
    val = 0
    for tau in range(t):  # For each day preceding the day of interest
        delay = t - tau  # The time period from each preceding day to the day of interest
        val += incidence[tau] * gen_time_densities[delay] * r0  # Increment the value for the day of interest
    incidence[t] = val

In [ ]:
incidence

Get rid of one loop to get lists/arrays for the incidence and generation time distribution

In [ ]:
for t in range(1, times):
    delays = [t - tau for tau in range(t)]
    gammas = gen_time_densities[delays]
    incidence[t] = (incidence[:t] * gammas).sum() * r0

In [ ]:
incidence

This is going to just keep going up exponentially, of course, because $R_{0} > 1$ and there is no susceptible depletion

In [ ]:
for t in range(1, times):
    incidence[t] = (incidence[:t] * gen_time_densities[t:0:-1]).sum() * r0

In [ ]:
incidence

In [ ]:
pd.Series(incidence).plot(labels={'index': 'day', 'value': 'incidence'})

Already some interesting phenomena there, 
in that the humps are the generations of cases from the first seeding infection,
which progressively smooth into one-another with generations of cases.
That's it for now.
Next step is that I'll try to incorporate the depletion of susceptibles.